# Convergence analysis of finite element solutions

This notebook draws heavily from the article [Axisymmetric formulation for elastic structures of revolution](https://comet-fenics.readthedocs.io/en/latest/demo/elasticity/axisymmetric_elasticity.html#Axisymmetric-formulation-for-elastic-structures-of-revolution) by J. Bleyer.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import mshr
import dolfin

In [ ]:
%matplotlib inline

## Parameters of the simulation

The cylinder is defined by its mean radius, $R$, and its thickness $h$. In cylindrical coordinates $(r, \theta, z)$, the domain is therefore defined by $R-h/2\leq r\leq R+h/2$.

In [ ]:
radius = 1.
thickness = radius/20.0

Change the following parameter if you wish to refine the mesh.

**@Corrado: please explain the `resolution` parameter in the `mshr.generate_mesh` function.**

In [ ]:
element_size = thickness/3
resolution = 100 # What's this?????

## Mesh generation

The `mshr` module uses the [Constructive Geometry](https://en.wikipedia.org/wiki/Constructive_solid_geometry) paradigm. The whole cross-section of the cylinder is therefore defined as the difference of the disk with radius $(r+h/2)$ and the disk with radius $(r+h/2)$.

Owing to symmetries, it is sufficient to model the upper-right quarter of the cross-section. We therefore subtract the top-left quadrant, and the rectangle made of the two lower quadrants.

In [ ]:
r_out = radius+0.5*thickness
r_in = radius-0.5*thickness

p0 = dolfin.Point(0., 0.)
p1 = dolfin.Point(-r_out, r_out)
p2 = dolfin.Point(-r_out, -r_out)
p3 = dolfin.Point(r_out, 0)

n = int(2*np.pi*r_out/element_size)

domain = (mshr.Circle(p0, r_out, n) -
          mshr.Circle(p0, r_in, n) -
          mshr.Rectangle(p0, p1) -
          mshr.Rectangle(p2, p3))
mesh = mshr.generate_mesh(domain, resolution)

In [ ]:
dolfin.plot(mesh);

## Boundary indicators

In [ ]:
tol = 1E-14

inner = dolfin.CompiledSubDomain('hypot(x[0], x[1]) <= r_in+tol', tol=tol, r_in=radius-0.5*thickness)
outer = dolfin.CompiledSubDomain('hypot(x[0], x[1]) >= r_out+tol', tol=tol, r_out=radius+0.5*thickness)

In [ ]:
class MySubDomain(dolfin.SubDomain):
    def __init__(self, tol=1E-14):
        self.tol = tol
        
class Inner(MySubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and dolfin.near(np.hypot(x[0], x[1]), r_in, self.tol)
    
class Outer(MySubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and dolfin.near(np.hypot(x[0], x[1]), r_out, self.tol)

class Left(MySubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and dolfin.near(x[0], 0., self.tol)

class Bottom(MySubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and dolfin.near(x[1], 0., self.tol)

In [ ]:
facets = dolfin.MeshFunction('size_t', mesh, 1)
facets.set_all(0)
Bottom().mark(facets, 1)
Outer().mark(faces, 2)
Left().mark(facets, 3)
Inner().mark(facets, 4)

## Strain-displacement operator

In [ ]:
x = dolfin.SpatialCoordinate(mesh)

In [ ]:
def eps(u):
    ε_rr = u[0].dx(0)
    ε_θθ = (u[1].dx(1)+u[0])/x[0]
    ε_θz = u[2].dx(1)/2./x[0]
    ε_zr = u[2].dx(0)/2.
    ε_rθ = (u[1].dx(0)+(-u[1]+u[0].dx(1))/x[0])/2.
    return dolfin.as_tensor([[ε_rr, ε_rθ, ε_zr],
                             [ε_rθ, ε_θθ, ε_θz],
                             [ε_zr, ε_θz, 0]])

In [ ]:
## Stress-strain operator

In [ ]:
E, nu = dolfin.Constant(1.), dolfin.Constant(0.3)
mu = E/2./(1.+nu)
lmbda = 2.*mu*nu/(1.-2.*nu)

In [ ]:
I2 = dolfin.Identity(3)

In [ ]:
def sig(epsilon):
    return  2.*mu*epsilon+lmbda*dolfin.tr(epsilon)*I2

## Variational formulation

In [ ]:
p_out = dolfin.Constant(0.)
p_in = dolfin.Constant(1.)

In [ ]:
element = dolfin.VectorElement('P', dolfin.triangle, degree=1, dim=3)
V = dolfin.FunctionSpace(mesh, element)
u = dolfin.TrialFunction(V)
v = dolfin.TestFunction(V)

In [ ]:
ds = dolfin.Measure("ds", subdomain_data=facets)

a = dolfin.inner(sig(eps(u)), eps(v))*dolfin.dx 
L = p_in*v[0]*ds(4)-p_out*v[0]*d(2)